In [ ]:
import logging
from typing import Dict
from initialize_rag import RAGInitializer

def process_query(query: str,
                 retriever,
                 reranker,
                 response_generator,
                 process_config: Dict,
                 send_nb_chunks_to_llm=2) -> Dict:
    """
    Process a single query through the complete RAG pipeline.
    
    This function orchestrates the query processing workflow:
    1. Optional query expansion
    2. Document retrieval
    3. Optional result reranking
    4. Response generation
    
    Args:
        query (str): The user's query
        retriever: Document retrieval component
        reranker: Result reranking component
        response_generator: Response generation component
        process_config (Dict): Processing configuration
        send_nb_chunks_to_llm (int): Number of chunks to send to LLM
        
    Returns:
        Dict: Processing results containing:
            - Query: Original query
            - Response: Generated response
            - Score: Best retrieval/reranking score
            
    Note:
        The function handles errors gracefully, returning an error message
        in the response if any step fails.
    """
    try:
        # Expand query if configured
        if process_config['retrieval']['use_query_expansion']:
            expanded_query = response_generator.expand_query(query)
            logging.info(f"Expanded query: {expanded_query}")
        else:
            expanded_query = query
            
        # Retrieve relevant documents using expanded or original query
        if process_config['retrieval']['use_bm25']:
            retrieved_results = retriever.retrieve_with_method(
                expanded_query,
                method="hybrid",
                top_k=process_config['retrieval']['top_k']
            )
        else:
            retrieved_results = retriever.retrieve_with_method(
                expanded_query,
                method="vector",
                top_k=process_config['retrieval']['top_k']
            )
        logging.info(f"Retrieved {len(retrieved_results)} documents")
        
        # Apply reranking if configured
        if process_config['retrieval']['use_reranking']:
            reranked_results = reranker.rerank(
                query,
                [r.document for r in retrieved_results],
                top_k=send_nb_chunks_to_llm
            )
            relevant_docs = [r.document for r in reranked_results]
            best_score = reranked_results[0].score if reranked_results else 0.0
            logging.info(f"Reranked results. Best score: {best_score}")
        else:
            relevant_docs = [r.document for r in retrieved_results]
            best_score = retrieved_results[0].score if retrieved_results else 0.0
            logging.info(f"Using retrieval scores. Best score: {best_score}")
        
        # Generate final response using selected documents
        response_data = response_generator.generate_answer(
            query,
            relevant_docs,
            metadata={'retrieval_score': best_score}
        )
        
        return {
            'Query': query,
            'Response': response_data['response'],
            'Score': best_score
        }
        
    except Exception as e:
        logging.error(f"Error processing query: {str(e)}")
        return {
            'Query': query,
            'Response': "An error occurred processing your query.",
            'Score': 0.0
        }

/home/t-bag/miniconda3/envs/rag_trustii/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
init_config_path = 'config/init_config.yaml'
process_config_path = 'config/process_config.yaml'

In [3]:
# Initialize system with both configs
initializer = RAGInitializer(init_config_path, process_config_path)
components = initializer.initialize()
components.retriever.initialize(components.original_chunks)

2024-11-10 02:47:21,541 - root - INFO - Initializing components...
2024-11-10 02:47:21,868 - datasets - INFO - PyTorch version 2.5.1+cu121 available.
2024-11-10 02:47:21,929 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: nomic-ai/nomic-embed-text-v1.5
2024-11-10 02:47:24,399 - transformers_modules.nomic-ai.nomic-bert-2048.c1b1fd7a715b8eb2e232d34593154ac782c98ac9.modeling_hf_nomic_bert - WARNING - <All keys matched successfully>
2024-11-10 02:47:26,272 - src.models.reranker - INFO - Initialized reranker model: BAAI/bge-reranker-v2-m3
2024-11-10 02:47:26,273 - root - INFO - Loading and preprocessing data...
2024-11-10 02:47:26,273 - src.data.data_ingestion - INFO - Loading CSV data...
2024-11-10 02:47:26,282 - src.data.data_ingestion - INFO - Loaded 2424 training samples and 1040 test samples
2024-11-10 02:47:26,282 - src.data.data_ingestion - INFO - Found 1 documents for processing
Loading documents: 100%|██████████| 1/1 [00:00<00:00, 222.98it/

#### --> IMPORTANT : You will need to restart the notebook each time in case of CUDA Out Of Memory

In [4]:
query = 'search_query: What is the subject key identifier of the DAC?'
# Process queries
result = process_query(query=query,
            retriever = components.retriever,
            reranker = components.reranker,
            response_generator = components.response_generator,
            process_config = components.process_config)

Batches: 100%|██████████| 1/1 [00:00<00:00,  8.36it/s]
2024-11-10 02:47:28,315 - root - INFO - Retrieved 50 documents
2024-11-10 02:47:28,315 - src.models.reranker - INFO - Reranking 50 documents for query: search_query: What is the subject key identifier of the DAC?
Reranking documents: 100%|██████████| 7/7 [00:03<00:00,  2.28it/s]
2024-11-10 02:47:31,380 - root - INFO - Reranked results. Best score: 4.782618522644043


In [5]:
print(result["Response"])

 The subject key identifier of the DAC is 96:C2:D9:24:94:EA:97:85:C0:D1:67:08:E3:88:F1:C0:91:EA:0F:D5.</s>
